In [ ]:
# make sure that mongo server is running 

In [ ]:
import os
from pymongo import MongoClient
import numpy as np
import utils.const as const

In [ ]:
db=const.grpId
collection=const.PostCollection

fileName='vgg16.csv'

post_out='vgg16_post.csv'
comment_out='vgg16_comment.csv'

outputLen=7*7*512

In [ ]:
client = MongoClient()
myCollect=client[db][collection]

In [ ]:
class Node:
  def __init__(self,length=outputLen):
    self.arr=np.zeros((length),dtype='float32')
    self.count=0
  
  def insert(self,arr2):
    arrT=self.arr.astype('float64')*self.count + arr2
    self.count+=1
    arrT/=self.count

    self.arr=arrT.astype('float32')
  
  def getString(self):
    ans=''
    for i in self.arr:
      ans+=','+str(i)
    
    return ans

In [ ]:
posts={}
comments={}
cc=0

for post in myCollect.find({}):
  if '_id' in post:
    posts[post['_id']]=Node()
    comments[post['_id']]={}

    if 'comments' in post:
      for comment in post['comments']:
        if '_id' in comment:
          cc+=1
          comments[post['_id']][comment['_id']]=Node()

print(len(posts))
print(cc)          


In [ ]:
def getVector(arr):
  length=len(arr)
  vector=np.zeros((length),dtype='float32')

  for i in range(length):
    vector[i]=float(arr[i])

  return vector.astype('float32')

In [ ]:
count=0
index=0

with open(fileName,'r') as fp:
  for line in fp:
    index+=1

    arr=line.split(',')
    if len(arr)<2:
      continue
    
    vector=getVector(arr[1:])

    arr2=arr[0].split('/')
    if len(arr2)<2:
      continue
    
    postId=arr2[1]

    if postId not in posts:
      raise Exception('a')

    if 'comments' == arr2[2]:
      commentId = arr2[3]
      count+=1
      if commentId not in comments[postId]:
        raise Exception('b')
      
      comments[postId][commentId].insert(vector)
    
    else:
      count+=1
      posts[postId].insert(vector)

In [ ]:
with open(post_out,'w') as fp:
  for k,v in posts.items():
    fp.write(k)
    fp.write(v.getString())
    fp.write('\n')

del posts

In [ ]:
with open(comment_out,'w') as fp:
  for k,v in comments.items():
    fp.write(k)
    for k2,v2 in v.items():
      fp.write(k2)
      fp.write(v2.getString())
      fp.write('\n')

del comments